# Calculating dividend tax and reporting it as a separate cash balance

In this Jupyter Notebook we'll see how to use LUSID to perform the following task:

**<div align="center">As a portfolio manager, I want LUSID to automatically calculate dividend tax for upserted transactions representing cash dividends, and report the tax amounts due in different currencies as separate cash balances.</div>**

In [ ]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
from IPython.core.display import HTML
from datetime import datetime, timedelta
import logging
logging.basicConfig(level=logging.INFO)

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusidjam import RefreshingToken
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.jupyter_tools import StopExecution
from finbourne_sdk_utils.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = SyncApiClientFactory(config_loaders=config_loaders)
    
# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

In [ ]:
# Create a scope and code to segregate data in this Notebook from others
module_scope = "FBNTutorials"
module_code = "DividendTaxV2"
print(f"'{module_scope}\{module_code}' scope and code created.")

In [ ]:
# Build all the required APIs
try:
    instruments_api = api_factory.build(la.InstrumentsApi)
    property_definition_api = api_factory.build(la.PropertyDefinitionsApi)
    transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)
    portfolios_api = api_factory.build(la.PortfoliosApi)
    taxruleset_api = api_factory.build(la.TaxRuleSetsApi)
    transaction_config_api = api_factory.build(la.TransactionConfigurationApi)
    print("All APIs built correctly")
except lu.ApiException as e:
    print(e)

## Setup: Delete any tax rule set from previous notebook run

This is because you can't have multiple matching tax rule sets with the same output property.

In [ ]:
try:
    delete_response=taxruleset_api.delete_tax_rule_set(
        scope = module_scope,
        code = module_code
    )
    print("Success!")
except lu.ApiException as e:
    #print(e)
    if json.loads(e.body)["name"] == "TaxRuleSetNotFound":
        logging.info(json.loads(e.body)["title"])

## 1. Master instruments and attach tax-related properties

Here we master BP and Microsoft as equity instruments in a custom instrument scope. Note GBP and USD currency instruments are pre-mastered in LUSID.

### 1.1 Create a property type for instrument tax domicile

We'll use properties of this type (in conjunction with a similar portfolio property) to trigger a tax rule set to determine the correct rate of dividend tax for transactions representing cash dividends in different jurisdictions.

In [ ]:
# Convenience function for creating property types
def create_property_type(property_domain, property_scope, property_code, data_type):
    
    # Define property type with a scope and code unique to the domain
    property_type_request = lm.CreatePropertyDefinitionRequest(
        domain = property_domain,
        scope = property_scope,
        code = property_code,
        display_name = property_code,
        data_type_id = lm.ResourceId(scope = "system", code = data_type)
    )
    
    # Create property type in LUSID
    try:
        property_type_response = property_definition_api.create_property_definition(
            create_property_definition_request = property_type_request
        )
        print(f"Property type created with the following key: {property_type_response.key}")
        return property_type_response.key
    except lu.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            logging.info(
                f"Property type with the following key already exists: {property_type_request.domain}/{property_type_request.scope}/{property_type_request.code}"
            )  
        return f"{property_type_request.domain}/{property_type_request.scope}/{property_type_request.code}"

# Create a property type representing the tax domicile of an instrument, and capture the 3-stage key
instrument_tax_property_key = create_property_type("Instrument", "DividendTax", "Country", "string")

### 1.2 Master instruments and attach tax domicile property values

In [ ]:
# Convenience function for mastering securities as equity instruments in a custom instrument scope
def master_instrument(figi, security, currency, domicile):
    
    # Define equity instrument
    instrument_request = {
        security: lm.InstrumentDefinition(
            name = security,
            identifiers = {"Figi": lm.InstrumentIdValue(value = figi)},
            definition = lm.Equity(instrument_type = "Equity", dom_ccy = currency),
            # Attach a property with a value declaring the tax domicile of the instrument
            properties = [
                lm.ModelProperty(
                    key = instrument_tax_property_key,
                    value = lm.PropertyValue(
                        label_value = domicile
                    )
                )
            ]
        )
    }
    
    # Upsert into LUSID
    instrument_response = instruments_api.upsert_instruments(
        request_body = instrument_request,
        scope = f"{module_scope}{module_code}"
    )

    # Transform upsert response to a dataframe and show internally-generated LUID identifier and tax-related property
    instrument_response_df = lusid_response_to_data_frame(list(instrument_response.values.values()))
    display(instrument_response_df[["name", "lusidInstrumentId"]])

# Master BP instrument with a tax-related property value of UK    
master_instrument("BBG000C05BD1", "BP", "GBP", "UK")

# Master Microsoft instrument with a tax-related property value of USA
master_instrument("BBG000BPH459", "Microsoft", "USD", "USA")

## 2. Set up a transaction portfolio

We need to:

* Attach a tax-related property to trigger a tax rule set to determine the correct rate of dividend tax, in conjunction with the instrument properties.
* Register a sub-holding key (SHK) so we can report dividend tax as a separate cash holding.
* Register a tax rule set scope to identify a set of applicable tax rule sets for the portfolio.
* Establish initial positions.


### 2.1 Create a property type for the portfolio's SHK

In [ ]:
# Create a SHK that enables us to report dividend tax as a separate holding, and capture the 3-stage key
sub_holding_key = create_property_type("Transaction", "SHKs", "DividendTax", "string")

### 2.2 Create a property type for the portfolio's tax domicile

In [ ]:
# Create a property type representing the tax domicile of a portfolio, and capture the 3-stage key
portfolio_tax_property_key = create_property_type("Portfolio", "DividendTax", "Domicile", "string")

### 2.3 Create the portfolio

In [ ]:
# Define transaction portfolio
portfolio_request=lm.CreateTransactionPortfolioRequest(
    display_name = f"Portfolio for dividend tax tutorial",
    code = f"{module_code}",
    # Set the portfolio currency
    base_currency = "GBP",
    # Must be before first transaction recorded
    created = to_date("2023-01-01"),
    # Attempt to resolve transactions to instruments in the custom scope before falling back to the default scope
    instrument_scopes = [f"{module_scope}{module_code}"],
    # Register the SHK with the portfolio
    sub_holding_keys = [sub_holding_key],
    # Register a tax rule set scope with the portfolio
    taxRuleSetScope=f"{module_scope}",
    # Attach a portfolio property with a value signifying the tax domicile
    properties={
        portfolio_tax_property_key: lm.ModelProperty(
            key = portfolio_tax_property_key,
            value = lm.PropertyValue(
                label_value = "GB"
            )
        )
    }
)

# Create transaction portfolio in LUSID
try:
    portfolio_response=transaction_portfolios_api.create_portfolio(
        scope = module_scope,
        create_transaction_portfolio_request = portfolio_request
    )
    # Confirm success
    print(f"Portfolio with display name '{portfolio_response.display_name}' created effective {str(portfolio_response.created)}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])

### 2.4 Confirm portfolio details

In [ ]:
try:
    transaction_portfolio_response = transaction_portfolios_api.get_details(scope = module_scope, code = module_code)
    tp_df = lusid_response_to_data_frame(transaction_portfolio_response)
    # Drop some noisy indices
    display(tp_df[~(tp_df.index.str.startswith('links') | tp_df.index.str.startswith('version') | tp_df.index.str.startswith('staged') | tp_df.index.str.startswith('href'))])
except lu.ApiException as e:
    print(e)

#### 2.4.1 Confirm the portfolio's tax-related property

The `GetDetails` API (above) has no `propertyKeys` parameter, but we can use the dedicated `GetPortfolioProperties` API instead.

In [ ]:
tp_property_response = portfolios_api.get_portfolio_properties(scope = module_scope, code = module_code)
tp_property_response_df = lusid_response_to_data_frame(list(tp_property_response.properties.values()))
tp_property_response_df

### 2.5 Establish positions in the portfolio

We'll establish positions effective 2 January so the portfolio has holdings before we perform tax-related operations.

In [ ]:
# Create convenience functions for adjusting holdings
def adjust_equity_holdings(figi, quantity, shareprice, ccy, ttpr):
    
    adjust_holdings_response = transaction_portfolios_api.batch_adjust_holdings(
        scope = module_scope,
        code = module_code,
        success_mode = "Atomic",
        request_body = {
            f"{figi}{quantity}{ccy}": lm.AdjustHoldingForDateRequest(
                effective_at = "2023-01-02",
                instrument_identifiers = {"Instrument/default/Figi": figi},
                tax_lots = [
                    lm.TargetTaxLotRequest(
                        units = quantity,
                        price = shareprice,
                        cost = lm.CurrencyAndAmount(amount = quantity * shareprice, currency = ccy),
                        # Trade to portfolio rate is 1 for GBP in a GBP-denominated portfolio
                        portfolio_cost = quantity * shareprice * ttpr
                    )
                ]
            )
        }
    )
    if len(adjust_holdings_response.failed) > 0:
        logging.info(adjust_holdings_response.failed)
    else:
        logging.info(f"Adjustment succeeded for {figi}")

def adjust_cash_holdings(quantity, ccy, ttpr):
    
    adjust_holdings_response = transaction_portfolios_api.batch_adjust_holdings(
        scope = module_scope,
        code = module_code,
        success_mode = "Atomic",
        request_body = {
            f"{quantity}{ccy}": lm.AdjustHoldingForDateRequest(
                effective_at = "2023-01-02",
                instrument_identifiers = {"Instrument/default/Currency": ccy},
                tax_lots = [
                    lm.TargetTaxLotRequest(
                        units = quantity,
                        # Local price of a currency is always 1
                        price = 1,
                        cost = lm.CurrencyAndAmount(amount = quantity * 1, currency = ccy),
                        # Trade to portfolio rate is 1 for GBP in a GBP-denominated portfolio
                        portfolio_cost = quantity * 1 * ttpr
                    )
                ]
            )
        }
    )
    if len(adjust_holdings_response.failed) > 0:
        logging.info(adjust_holdings_response.failed)
    else:
        logging.info(f"Adjustment succeeded for {ccy}")

In [ ]:
# Set starting position for BP
adjust_equity_holdings("BBG000C05BD1", 1000, 10, "GBP", 1)
# Set starting position for Microsoft
adjust_equity_holdings("BBG000BPH459", 1000, 10, "USD", 0.8)

# Set starting position for GBP     
adjust_cash_holdings(20000, "GBP", 1)
# Set starting position for USD
adjust_cash_holdings(20000, "USD", 0.8)

### 2.6 Confirm positions after they have settled

In [ ]:
# Convenience function for generating a holdings report
def get_portfolio_holdings(holdings_date):
    
    get_holdings_response = transaction_portfolios_api.get_holdings(
        scope = module_scope, 
        code = module_code,
        # Retrieve properties to make results more intuitive
        property_keys = ["Instrument/default/Name", portfolio_tax_property_key, instrument_tax_property_key],
        effective_at = to_date(holdings_date).isoformat()
    )
    
    # Transform API response to a Pandas dataframe and show it
    get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
    # Drop some noisy columns
    if len(get_holdings_response.values) > 0:
        get_holdings_response_df = get_holdings_response_df.loc[:, ~get_holdings_response_df.columns.str.startswith('properties') & ~get_holdings_response_df.columns.str.startswith('subHoldingKeys') & ~get_holdings_response_df.columns.str.startswith('version')]

    display(get_holdings_response_df)
    
get_portfolio_holdings("2023-01-04")

### 2.7 Audit output transactions

LUSID automatically generates 'output transactions' under-the-hood to enrich manually-upserted 'input transactions' (and virtual economic activity such as corporate actions) with extra information.

Here we use a window of 2-4 January to cover the 4 input transactions upserted above; we should see 4 output transactions.

In [ ]:
# Create convenience function for generating output transactions for a particular window
def get_output_transactions(start, end, extra_properties, transpose):
    
    output_transactions_response = transaction_portfolios_api.build_transactions(
        scope = module_scope, 
        code = module_code,
        transaction_query_parameters = lm.TransactionQueryParameters(
            start_date = to_date(start).isoformat(),
            end_date = to_date(end).isoformat()
        ),
        # Retrieve property to make results more intuitive
        property_keys = ["Instrument/default/Name", extra_properties, "Transaction/system/AppliedTaxRule"]
    )
    
    if transpose == "vertical":
        output_transactions_response_df = lusid_response_to_data_frame(output_transactions_response).transpose()
    else:
        output_transactions_response_df = lusid_response_to_data_frame(output_transactions_response)
    display(output_transactions_response_df)
    
get_output_transactions("2020-01-02", "2023-01-04", "", "horizontal")

## 3. Create a tax rule set to calculate different rates of dividend tax

A tax rule set has:

* A `scope` that must be registered with each portfolio you wish tax calculations to impact, and a `code` unique among all tax rule sets in the scope.
* Any number of tax rules, each with any number of match criteria that are processed in order. Note values can be compared against properties in the `Instrument`, `Portfolio` and `InstrumentEvent` domains, and also SHKs in the `Transaction` domain if the `criterion_type` is set to `SubHoldingKeyValueEquals`. Tax rules apply from 'now' unless you explicitly specify an `effectiveAt`.
* An 'output property' in the `Transaction` domain that stores the result; that is, the amount of tax due for each matching transaction, calculated using the appropriate rate. Note the name of the output property must be unique among all tax rule sets in the scope.

### 3.1 Create an 'output' property type to store the calculated amount of tax due

In [ ]:
# Create a property type representing the amount of dividend tax due, and capture the 3-stage key
outputtransaction_taxdue_property_key = create_property_type("Transaction", "DividendTax", "AmountDueV2", "number")

### 3.2 Create a tax rule set handling different UK and US rates of dividend tax effective 1 Jan 2023

We specify an explicit `effectiveAt` so the tax rules in our set apply from a specific date, rather than 'now'. 

These rules then stay in effect until we call the `UpdateTaxRuleSet` API to replace them with rules (presumably with different rates) that have a subsequent `effectiveAt`.

In [ ]:
def create_taxruleset(effective_date, uk_tax_rate, us_tax_rate):
    
    # Define tax rule set
    tax_request = lm.TaxRuleSet(
        id = lm.ResourceId(
            scope = module_scope,
            code = module_code
        ),
        description = "Dividend tax for UK-domiciled portfolios",
        display_name = "Dividend tax",
        # Specify output property to store the amount of tax due
        output_property_key = outputtransaction_taxdue_property_key,
        rules = [
            lm.TaxRule(
                name = "UKDividendTax",
                description = "Dividend tax that applies to UK equities",
                rate = uk_tax_rate,
                match_criteria = [
                    lm.PropertyValueEquals(
                        criterion_type = "PropertyValueEquals",
                        property_key = instrument_tax_property_key,
                        value = "UK" 
                    ),
                    lm.PropertyValueEquals(
                        criterion_type="PropertyValueEquals",
                        property_key = portfolio_tax_property_key,
                        value = "GB" 
                    )
                ],
            ),
            # Create rule for US dividend tax in a GB-domiciled portfolio for 2023 tax year
            lm.TaxRule(
                name = "USDividendTax",
                description = "Dividend tax that applies to US equities in UK portfolios",
                rate = us_tax_rate,
                match_criteria = [
                    lm.PropertyValueEquals(
                        criterion_type = "PropertyValueEquals",
                        property_key = instrument_tax_property_key,
                        value = "USA" 
                    ),
                    lm.PropertyValueEquals(
                        criterion_type = "PropertyValueEquals",
                        property_key = portfolio_tax_property_key,
                        value = "GB" 
                    )
                ],
            )
        ]
    )
    
    # Create tax rule set in LUSID
    try:
        tax_response = taxruleset_api.create_tax_rule_set(
            create_tax_rule_set_request = tax_request,
            effective_at = to_date(effective_date).isoformat()
        )
        print("Tax rule set created")
        #print(tax_response)
    except lu.ApiException as e:
        if json.loads(e.body)["name"] == "TaxRuleSetAlreadyExists":
            logging.info(
                "Tax rule set already exists."
            )
        else:
            print(e)
        
create_taxruleset("2023-01-01", 0.25, 0.1)

### 3.3 Confirm tax rule set details

If we specify any `effectiveAt` date later than 1 Jan 2023 we should see our tax rules. Specifying an earlier date returns an empty `rules[]` list.

In [ ]:
def get_taxruleset(from_date):

    tax_response = taxruleset_api.get_tax_rule_set(module_scope, module_code, effective_at=from_date)
    tax_response_df = lusid_response_to_data_frame(tax_response)
    # Drop some noisy indices
    display(tax_response_df[~(tax_response_df.index.str.startswith('links') | tax_response_df.index.str.startswith('version'))])

# Specify an empty string ("") for 'now'...
get_taxruleset("2022-01-01") # 2022; rules shouldn't exist
get_taxruleset("2023-01-01") # 2023; rules should exist

## 4. Create a custom transaction type to trigger the tax rule set and report tax amounts due

We need our custom transaction type to do three things:

1. Trigger the tax rule set to calculate dividend tax and return the amount of tax due for each transaction to which the type applies. To do this we'll add a calculation type.
2. Add the gross dividend payment to the main cash balance in the appropriate currency. To do this we'll add a movement using one of the built-in sides provided with LUSID.
3. Report the dividend tax amount due as a separate cash balance. To do this we'll add a movement using a custom side that maps the tax rule set's output property to the portfolio's SHK.

### 4.1 Create a custom side

We first need to create a custom side with the `units` and `amount` fields  set to the value of the tax rule set's output property, to capture the amount of dividend tax due.

In [ ]:
# Define custom side
side_request = lm.SideDefinitionRequest(
    security = "Txn:SettleCcy",
    currency = "Txn:SettlementCurrency",
    rate = "Txn:TradeToPortfolioRate",
    units = outputtransaction_taxdue_property_key,
    amount = outputtransaction_taxdue_property_key,
)

# Create custom side in LUSID
try:
    side_response = transaction_config_api.set_side_definition(
        # Specify the name of the custom side
        side = f"{module_scope}{module_code}Side",
        side_definition_request = side_request
    )
    side_df = lusid_response_to_data_frame(side_response)
    # Drop some noisy indices
    display(side_df[~(side_df.index.str.startswith('links'))])
    #display(side_df)
except lu.ApiException as e:
    print(e)
    if json.loads(e.body)["name"] == "InvalidParameterValue":
        logging.info("Side definition already exists.")

### 4.2 Create a custom transaction type using the custom side

Our custom transaction type has three movements:

* The first movement adds the gross dividend payment to the main cash balance.
* The second movement maps the output property captured by the custom side to the portfolio's SHK, to report it as a separate cash holding.
* The third movement is unrelated to the task at hand but is important to report transactions representing cash dividends properly in downstream A2B reports and trial balances.

Setting the calculation type to `TaxAmounts` triggers the tax rule set. Details of the equity instrument are passed in using the built-in `Side1` designed for securities (that is, non-currency instruments). 

In [ ]:
# Define custom transaction type
transaction_type_request = lm.TransactionTypeRequest(
    # Create an alias
    aliases = [
        lm.TransactionTypeAlias(          
            type = f"{module_scope}{module_code}TransactionType",
            description = "Calculating and applying dividend tax as a separate cash holding",
            transaction_class = "Basic",
            transaction_roles = "AllRoles"
        )
    ],
    calculations = [
        lm.TransactionTypeCalculation(
            type = "TaxAmounts",
            side = "Side1"
        )
    ],
    movements = [
        # Create a positive movement that increases the main cash balance by the gross dividend payment
        lm.TransactionTypeMovement(
            name = "Add dividend to main cash balance",
            movement_types = "CashAccrual",
            direction = 1,
            side = "Side2",
        ),
        # Create a negative movement that uses the custom side to report the tax amount due as a separate cash balance
        lm.TransactionTypeMovement(
            name = "Report dividend tax as a separate cash holding",
            movement_types = "CashReceivable",
            direction = -1,
            # Capture the amount of tax due from the tax rule set's output property
            side = f"{module_scope}{module_code}Side",
            # Map the result of the movement to the portfolio's SHK, to report it separately
            mappings = [
                lm.TransactionPropertyMappingRequest(
                    property_key = f"{sub_holding_key}",
                    set_to = "DividendTax",
                )
            ],
        ),
        # Create a movement that handles Carry activity properly
        lm.TransactionTypeMovement(
            name = "Report the dividend as a flow out of the investment",
            movement_types = "Carry",
            direction = 1,
            side = "Side1",
        )
    ]
)

# Create custom transaction type in LUSID    
try:
    transaction_type_response = transaction_config_api.set_transaction_type(
        source = "default",
        # Specify the primary alias name (in this case, the name of the only alias in the transaction type)
        type = f"{module_scope}{module_code}TransactionType",
        transaction_type_request = transaction_type_request
    )
    tt_df = lusid_response_to_data_frame(transaction_type_response)
    # Drop some noisy indices
    display(tt_df[~(tt_df.index.str.startswith('links'))])
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "TransactionTypeDuplication":
        logging.info(f"Transaction type already exists.")

## 5. Load transactions representing cash dividends

We'll load two transactions on 20 September, one representing a cash dividend for BP and the other a cash dividend for Microsoft. Both trades settle two days later.

Since a cash dividend represents a flow (of value) out of a security, each transaction is recorded 'in' the equity instrument rather than the currency itself. The `units` of the transaction is the number of shares held and the `total consideration.amount` is the number of shares multipled by the dividend.

In our example, we hold 1000 units of BP and Microsoft, and they both pay a 50 pence/cent cash dividend, resulting in 500 units of currency as the gross dividend amount.

Note the transactions are set to use the custom transaction type.

In [ ]:
# Convenience function for upserting transactions to a portfolio
def load_transaction(txnid, tttype, instrid, quantity, priceorrate, ccy, date, ttpr):
    
    if instrid[0:4] == "CCY_":
        identifier = {"Instrument/default/LusidInstrumentId": instrid}
    else:
        identifier = {"Instrument/default/Figi": instrid}
    
    create_txn_request = {
        instrid: lm.TransactionRequest(
            transaction_id=txnid,
            type=tttype,
            instrument_identifiers = identifier,
            transaction_date=to_date(date).isoformat(),
            settlement_date=(to_date(date) + timedelta(days = 2)).isoformat(),
            units=quantity,
            transaction_price = lm.TransactionPrice(
                price=priceorrate,
                type="Price"
            ),
            total_consideration = lm.CurrencyAndAmount(
                amount = quantity * priceorrate,
                currency = ccy,
            ),
            properties={
                "Transaction/default/TradeToPortfolioRate": lm.PerpetualProperty(
                    key = "Transaction/default/TradeToPortfolioRate",
                    value = lm.PropertyValue(
                        metric_value = lm.MetricValue(
                            value = ttpr,
                        )
                    )
                )
            }
        )
    }
    
    #Upsert to LUSID
    create_txn_response = transaction_portfolios_api.batch_upsert_transactions(
        scope = f"{module_scope}",
        code = module_code,
        success_mode="Partial",
        request_body = create_txn_request
    )
    
  
    # Transform upsert response to a dataframe
    create_txn_response_df = lusid_response_to_data_frame(list(create_txn_response.values.values()))
    display(create_txn_response_df)

In [ ]:
# BP
load_transaction("ManualTxn01", f"{module_scope}{module_code}TransactionType", "BBG000C05BD1", 1000, 0.5, "GBP", "2023-09-20", 1)
# MSFT
load_transaction("ManualTxn02", f"{module_scope}{module_code}TransactionType", "BBG000BPH459", 1000, 0.5, "USD", "2023-09-20", 0.8)

## 6. Generate a holdings report

### 6.1 On the trade date

The gross dividend amounts are reported using the (unsettled) holding type of `Accrual`, while the tax amounts due are classified separately using the `DividendTax` SHK and reported using the (unsettled) holding type of `Receivable`.

In [ ]:
get_portfolio_holdings("2023-09-20")

### 6.2 On the settlement date

The gross dividend payments are added to the main cash balances, but the dividend tax amounts remain classified separately using the SHK.

In [ ]:
get_portfolio_holdings("2023-09-22")

### 6.3 Audit output transactions

Here we can use a window covering the trade and settlement dates to see the output transactions that LUSID generated under-the-hood to enrich these two input transactions.

In [ ]:
get_output_transactions("2023-09-20", "2023-09-22", outputtransaction_taxdue_property_key, "vertical")